* 文字列データを表形式データに変換するコード
* 出力されたコードをコードモデルに組み込むコード

In [3]:
import sys
sys.path.append('..')

from src.ibldataset.psuedodata import jinja_rendering, df_to_string

from datasets import load_dataset

dataset_name = 'ibl-regression-ver2-linear'

In [7]:
code = """
if row["B"] < 18:
    y = 8*row["C"]
else:
    y = 4*row["C"]
"""

In [8]:
code_model = jinja_rendering(code)
print(code_model)

def predict(x):
    df = x.copy()
    output = []
    for index, row in df.iterrows():
        
        if row["B"] < 18:
            y = 8*row["C"]
        else:
            y = 4*row["C"]
        
        output.append(y)
    return np.array(output)


In [9]:
test_dataset = load_dataset(f"fuyu-quant/{dataset_name}", split='test')

In [10]:
i = 1
data_str = test_dataset[i]['instruction']

In [ ]:
def ibl_output(model, input):
    prompt = f"### Instruction: {input}\n\n### Response: "

    # 推論の実行
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100
            )
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    output = output.split("### Response: ")[-1]
    return output

In [11]:
import pandas as pd
import numpy as np

def str_to_df(data_str):
    lines = data_str.split('\n')
    data = [line.split() for line in lines]
    df = pd.DataFrame(data[1:], columns=data[0])
    df = df.astype(int)
    return df

def code_model_pred(df, code_model):
    df = df.drop(['y'], axis=1)
    local_vars = {}
    exec(code_model, globals(), local_vars)
    df['y'] = local_vars['predict'](df)
    return df

In [12]:
df = str_to_df(data_str)
pred_df = code_model_pred(df, code_model)
str_df = pred_df.to_string(index=False)
str_df